In [ ]:
from fair import FAIR

In [ ]:
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import copy

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
from tqdm.auto import tqdm

In [ ]:
from scipy.stats import linregress

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1850, 2050, 1)

In [ ]:
scenarios = ['esm-bell-750PgC', 'esm-bell-1000PgC', 'esm-bell-2000PgC']

In [ ]:
# do one at a time
#scenarios = [scenarios[0]]
f.define_scenarios(scenarios)
fair_params_df = pd.read_csv('../data/ar6_calibration_ebm3.csv', index_col=0)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
f.concentration.loc[dict(specie='CH4')] = 808.2490285
f.concentration.loc[dict(specie='N2O')] = 273.021047

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# Scale CO2 forcing based on its 4xCO2 calibration
calibrated_f4co2_mean = fair_params_df.loc[:,'F_4xCO2'].values.mean()
fill(
    f.species_configs['forcing_scale'], 
    1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[:,'F_4xCO2'].values)/calibrated_f4co2_mean,
    specie='CO2'
)

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], 284.3169988, specie='CO2')

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
f.forcing

In [ ]:
pl.plot(f.temperature[100:,0,:,0] - f.temperature[100,0,:,0]);

In [ ]:
pl.plot(f.temperature[100:,1,:,0] - f.temperature[100,1,:,0]);

In [ ]:
pl.plot(f.temperature[100:,2,:,0] - f.temperature[100,2,:,0]);

In [ ]:
pl.hist(f.temperature[150,0,:,0] - f.temperature[100,0,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[150,1,:,0] - f.temperature[100,1,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[150,2,:,0] - f.temperature[100,2,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
print(np.percentile(f.temperature[150,0,:,0] - f.temperature[100,0,:,0], (5,50,95)))
print(np.percentile(f.temperature[150,1,:,0] - f.temperature[100,1,:,0], (5,50,95)))
print(np.percentile(f.temperature[150,2,:,0] - f.temperature[100,2,:,0], (5,50,95)))

In [ ]:
pl.hist(f.temperature[200,0,:,0] - f.temperature[100,0,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[200,1,:,0] - f.temperature[100,1,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
pl.hist(f.temperature[200,2,:,0] - f.temperature[100,2,:,0], alpha=0.4, bins=np.arange(-0.8, 1.1, 0.02))
print(np.percentile(f.temperature[200,0,:,0] - f.temperature[100,0,:,0], (5,50,95)))
print(np.percentile(f.temperature[200,1,:,0] - f.temperature[100,1,:,0], (5,50,95)))
print(np.percentile(f.temperature[200,2,:,0] - f.temperature[100,2,:,0], (5,50,95)))

In [ ]:
pl.plot(f.concentration[:,0,:,0]);